In [1]:
import os
import pandas as pd

from langchain_anthropic import ChatAnthropic

from logger import logger
from modules import extractor

C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_9116\4073229117.py:7: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from modules import extractor


to access modules in parent directory

In [2]:
import sys

# Add the parent directory (my_app/) to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

________________________________________________________

In [3]:
logger.info("-------------- New Notebook Session --------------")

2025-02-25 09:04:34,678 - LOGZ - INFO - -------------- New Notebook Session --------------


In [4]:
with open("../api_key.txt") as f:
    api_key=f.read()

llm = ChatAnthropic(model="claude-3-5-sonnet-20241022",
                    temperature=0,
                    max_tokens=512,
                    timeout=None,
                    max_retries=2,
                    api_key=api_key)

In [5]:
def process_folder(folder_path, **kwargs):
    if not os.path.exists(folder_path):
        logger.error("Error: Folder {folder_path} does not exist.")
        return

    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

    if not pdf_files:
        logger.info("No PDF files found.")
        return

    logger.info(f"Found {len(pdf_files)} PDF(s). Processing...\n")

    results = []

    for pdf in pdf_files:
        pdf_path = os.path.join(folder_path, pdf)
        df = extractor.process_pdf(pdf_path, **kwargs)
        results.append(df)

    logger.info("Finished processing all PDFs.")
    return results

In [6]:
folder = '../Data/AllPDF'

args = {
    "llm": llm,
}

results = process_folder(folder, **args)

2025-02-25 09:04:34,712 - LOGZ - INFO - Found 3 PDF(s). Processing...

2025-02-25 09:04:34,713 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\TPC Rule Manual.pdf
2025-02-25 09:04:35,411 - LOGZ - INFO - Starting extraction on 1 chunks.
2025-02-25 09:04:40,255 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\TX PPA APCIC Rules 101518RB.pdf
2025-02-25 09:04:40,981 - LOGZ - INFO - Starting extraction on 3 chunks.
2025-02-25 09:04:45,918 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\TX RV 201401 Rules.pdf
2025-02-25 09:04:46,202 - LOGZ - INFO - Starting extraction on 2 chunks.
2025-02-25 09:04:52,214 - LOGZ - INFO - Finished processing all PDFs.


In [7]:
iterator = iter(results)
final_df = next(iterator)

try:
    while True:
        df = next(iterator)
        final_df = pd.concat([final_df, df], ignore_index=True)
except StopIteration:
    print("End of list reached")

final_df

End of list reached


,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,territory_factor_BI
0,"Tesla Property & Casualty, Inc.",100,"The minimum premium is for a six-month policy,...",6,Policies may be written for policy periods for...,1
1,Allied Property and Casualty Insurance Company,null,null,6,"A Policy Period is six or twelve months, as of...",null
2,Allied Property and Casualty Insurance Company,null,No minimum premium information found in the text,null,No explicit policy period mentioned in the text,The territory factors are listed by county and...
3,Allied Property and Casualty Insurance Company,,None,6,Premiums are developed based on a six-month po...,
4,Progressive County Mutual Insurance Company,null,While Rule G05 mentions there is a Minimum Wri...,12,"According to Rule G03: ""All policies are writt...",null
5,Progressive County Mutual Insurance Company,,None,35,Based on the chargeable period mentioned in Ru...,


In [8]:
final_df.to_csv("extracted.csv", index=False)

how the hell am i going to handle tables ?